Estrutura dos arquivos:

src
├── libs
│   ├── dataframe.py
│   ├── conexaoDB.py
│   ├── criarTabelas.py
│   ├── fraudes.py
├── files
│   ├── csv_clientes
│   ├── csv_transacoes
├── main.py


inicializando pyspark e criando uma sessão

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Teste").getOrCreate()

CRIANDO DATAFRAMES

In [39]:
from pyspark.sql.functions import col

class Data():
    '''A classe Dataframe tem o objetivo de proporcionar métodos capazes de ler arquivos CSVs, gerar dataframes e modificá-los.'''
     
    def __init__(self, csv_path,tipo):
        #self.spark = SparkHelper.getInstance().spark
        self.csv_path = csv_path
        self.tipo = tipo
        #self.spark = SparkSession.builder.master("local[*]").appName("b").getOrCreate()
        
    def criar(self):
        '''O método criar procura ler uma pasta que contém arquivo CSV gerando um dataframe.'''
        self.path_completo = self.csv_path + self.tipo + '-0**.csv'
        print(self.path_completo)
        
        #Lendo o arquivo CSV bruto
        try:
            self.df_bruto = spark.read.csv(self.path_completo, sep=";", header=False, inferSchema=True)
        except AnalysisException:
            print('O path nao existe. Por favor, insira um path valido.\nPode ser que o "tipo" passado nao seja "clients", "transaction_in" ou "transaction_out".')
            return 0
            
        # Filtrando o header. Note que está considerando que se tiverem vários headers, apenas 1 será selecionado
        self.header = self.df_bruto.filter(col("_c0") == "id").limit(1)

        # Excluindo o/os header do Dataframe bruto
        df_filtrado = self.df_bruto.where((col("_c0") != "id"))

        # Convertendo a primeira linha do header em uma lista de strings
        header_columns = self.header.first()

        # Gerando um dataframe com o header certo
        self.df = df_filtrado.toDF(*header_columns)
        print(self.df.count())

        return self.df
    

TESTANDO NA CLASSE - CLIENTES

In [40]:
csv_path = '/home/jovyan/app/files/'
csv_path2 = '/home/jovyan/app/files'

df_clientes1 = Data(csv_path,'client').criar()
#df_clientes1.show()


/home/jovyan/app/files/client-0**.csv


AnalysisException: Path does not exist: file:/home/jovyan/app/files/client-0**.csv

testando sem ser na classe

In [3]:
tipo = 'clients'
csv_path = '/home/jovyan/app/files/'
teste = csv_path + tipo + '-0**.csv'
print(teste)
try:
    df_bruto = spark.read.csv('/home/jovyan/app/files/clients-0**.csv', sep=";", header=False, inferSchema=True) 
except:
    print('O tipo passado nao eh "clients", "transaction_in" ou "transaction_out".')
print(df_bruto)
df_bruto.show()

/home/jovyan/app/files/clients-0**.csv
DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]
+---+--------------------+--------------------+--------------------+----------------+
|_c0|                 _c1|                 _c2|                 _c3|             _c4|
+---+--------------------+--------------------+--------------------+----------------+
| 55|   Edmilson da silva|edmilson-da-silva...|2019-08-29 21:54:...|+55(22)2922-2626|
| 78|Maxson Barros do ...|maxson-barros-do-...|2019-09-09 23:03:...|+55(22)2126-2529|
| 61| Bruno cesar e silva|bruno-cesar-e-sil...|2019-08-29 22:05:...|+55(23)2528-2729|
|106|Dernival passos d...|dernival-passos-d...|2019-09-26 23:40:...|+55(29)2927-2322|
|107| José rubian de goes|jose-rubian-de-go...|2019-09-28 10:09:...|+55(22)2023-2620|
|108|Angelica Dos Sant...|angelica-dos-sant...|2019-09-28 11:48:...|+55(20)2521-3030|
|109|          Alanderson|alanderson_109@gm...|2019-09-28 21:15:...|+55(22)2323-2426|
|142|      Silvio gabriel|

TESTANDO NA CLASSE - TRANSACOES

In [40]:
header_transacoes_in_path = "/home/jovyan/app/files/csv_transacoes/transacoes_in/header-transaction-in-001.csv"
transacoes_in_path = "/home/jovyan/app/files/csv_transacoes/transacoes_in/transaction-in-00*.csv"

df_transacoes_in = Dataframe(transacoes_in_path).criar(header_transacoes_in_path)

df_transacoes_in.show()

df_transacoes_in.createOrReplaceTempView("TransacoesIn")
df_transacoes_in
# Executa uma consulta SQL para filtrar as fraudes

fraudes = spark.sql("SELECT * FROM TransacoesIn GROUP BY cliente_id")
                    #data_cadastro LIKE '2021-03-28 %:00:%'")

# Exibe o resultado
fraudes.show()


+----+----------+-----+-------------------+
|  id|cliente_id|valor|               data|
+----+----------+-----+-------------------+
|8615|       586|  0.2|2022-01-19 20:12:26|
|8613|       586|  0.2|2022-01-19 20:11:25|
|8611|       586|  0.2|2022-01-19 20:10:05|
|8606|       910|300.0|2022-01-19 19:59:36|
|8604|        76|100.0|2022-01-18 12:48:14|
|8603|        76|100.0|2022-01-18 12:48:04|
|8602|        76|100.0|2022-01-18 12:47:47|
|8601|        76|100.0|2022-01-18 12:47:43|
|8600|        76|100.0|2022-01-18 12:47:39|
|8599|        76|100.0|2022-01-18 12:43:05|
|8598|        76|100.0|2022-01-18 12:42:56|
|8597|        76|100.0|2022-01-18 12:40:28|
|8596|        76|100.0|2022-01-18 12:38:19|
|8595|        76|100.0|2022-01-18 12:37:59|
|8594|        76|100.0|2022-01-18 12:37:29|
|8593|        76|100.0|2022-01-18 12:37:19|
|8592|       907| 10.0|2022-01-18 12:30:26|
|8591|       907| 10.0|2022-01-18 12:30:14|
|8590|       907| 10.0|2022-01-18 12:30:10|
|8589|       907| 10.0|2022-01-1

AnalysisException: expression 'transacoesin.id' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;
Aggregate [cliente_id#2365], [id#2364, cliente_id#2365, valor#2366, data#2367]
+- SubqueryAlias transacoesin
   +- View (`TransacoesIn`, [id#2364,cliente_id#2365,valor#2366,data#2367])
      +- Union false, false
         :- Relation [id#2364,cliente_id#2365,valor#2366,data#2367] csv
         +- Relation [_c0#2389,_c1#2390,_c2#2391,_c3#2392] csv


In [58]:
df.show()

+---+--------------------+--------------------+-------------------+----------------+
| id|                nome|               email|      data_cadastro|        telefone|
+---+--------------------+--------------------+-------------------+----------------+
|641|Priscila Felix do...|priscila-felix-do...|2021-03-28 18:46:57|+55(30)2227-2428|
| 94|             idelmon|idelmon_94@gmail.com|2019-09-19 12:33:19|+55(29)3027-2026|
|584|Liliane soares da...|liliane-soares-da...|2021-02-10 19:15:30|+55(21)2024-2520|
|580|Fagner jose dos s...|fagner-jose-dos-s...|2021-02-07 01:47:04|+55(24)2624-2029|
| 21|               Cildo|  cildo_21@gmail.com|2019-07-30 11:40:10|+55(21)2222-2422|
|582|Nielton da Silva ...|nielton-da-silva-...|2021-02-09 00:11:22|+55(27)2028-2828|
|586|Armando Teles da ...|armando-teles-da-...|2021-02-12 15:20:14|+55(27)2720-2230|
|151|            Fabricio|fabricio_151@gmai...|2019-10-14 21:16:27|+55(20)2121-2326|
| 83|       Flavio junior|flavio-junior_83@...|2019-09-11 15:24:0

TESTANDO SEM ESTAR NA CLASSE

In [26]:
from pyspark.sql.functions import col

# Lendo o arquivo CSV bruto
df_bruto = spark.read.csv("/home/jovyan/app/files/csv_clientes/", sep=";", header=False, mode="DROPMALFORMED")

# Filtrando o header. Note que está considerado que se tiverem vários headers, apenas 1 será selecionado
header = df_bruto.filter(col("_c0") == "id").limit(1)

# Excluindo o/os header do Dataframe bruto
df_filtrado = df_bruto.where((col("_c0") != "id"))

# Convertendo a primeira linha do header em uma lista de strings
header_columns = header.first()

# Gerando um dataframe com o header certo
df = df_filtrado.toDF(*header_columns)

# Exibindo o resultado
print(df.count())
df.show()

404
+---+--------------------+--------------------+--------------------+----------------+
| id|                nome|               email|       data_cadastro|        telefone|
+---+--------------------+--------------------+--------------------+----------------+
| 55|   Edmilson da silva|edmilson-da-silva...|2019-08-29 21:54:...|+55(22)2922-2626|
| 78|Maxson Barros do ...|maxson-barros-do-...|2019-09-09 23:03:...|+55(22)2126-2529|
| 61| Bruno cesar e silva|bruno-cesar-e-sil...|2019-08-29 22:05:...|+55(23)2528-2729|
|106|Dernival passos d...|dernival-passos-d...|2019-09-26 23:40:...|+55(29)2927-2322|
|107| José rubian de goes|jose-rubian-de-go...|2019-09-28 10:09:...|+55(22)2023-2620|
|108|Angelica Dos Sant...|angelica-dos-sant...|2019-09-28 11:48:...|+55(20)2521-3030|
|109|          Alanderson|alanderson_109@gm...|2019-09-28 21:15:...|+55(22)2323-2426|
|142|      Silvio gabriel|silvio-gabriel_14...|2019-10-04 09:47:...|+55(22)3026-2123|
|144|JOSÉ CARLOS DA SI...|jose-carlos-da-si...|201